In [0]:
%load_ext autoreload
%autoreload 2

In [2]:
pwd

'/content'

In [3]:
# for Google Colab only: Mount my Google Drive
from google.colab import drive
drive.mount('/gdrive')
#%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
# for Google Colab only: Navigate to my snaug project folder
#cd 'My Drive/Colab Notebooks/cavy_breed_clf'

In [0]:
from pathlib import Path

In [0]:
proj_base = Path('/gdrive/My Drive/Colab Notebooks/cavy_breed_clf/')

In [0]:
proj_data = proj_base/'data'
proj_lib = proj_base/'lib'
proj_images = proj_base/'images'

In [0]:
!mkdir images

In [0]:
!cp -r "{proj_images}" .

In [9]:
!ls -al images

total 88
drwxr-xr-x 7 root root  4096 Dec 22 21:07 .
drwxr-xr-x 1 root root  4096 Dec 22 20:55 ..
drwx------ 2 root root 28672 Dec 22 21:05 Abyssinian
drwx------ 2 root root 20480 Dec 22 20:59 American
drwx------ 2 root root  4096 Dec 22 21:08 new_test
drwx------ 2 root root 12288 Dec 22 21:01 Silkie
drwx------ 2 root root 12288 Dec 22 21:07 Skinny


In [10]:
import sys
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython']

In [11]:
#sys.path.append("/gdrive/My Drive/Colab Notebooks/cavy_breed_clf/lib")
sys.path.append(str(proj_lib))
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/gdrive/My Drive/Colab Notebooks/cavy_breed_clf/lib']

In [12]:
from data_common import (target_names, Read_and_Process_Image, 
                             Load_and_Split)
from ml_common import (Vanilla_ML_Run, Vanilla_ML_Predict, 
                           Vanilla_ML_Run_CV)
from nn_common import (Image_CNN_Multilayer, Image_CNN_Multilayer_Train,
                           Image_CNN_From_InceptionV3,
                           Image_CNN_From_InceptionV3_Train, Image_NN_Predict,
                           Image_NN_Predict_One, Image_NN_Predict_Random_Test_Images,
                           Image_NN_Plt_Acc, Image_NN_Plt_Loss, 
                           Image_NN_Plt_Training, Image_NN_Plt_Validation,
                           Save_NN_Model_Data, Load_NN_Model_Data)
from ml_common import (Plot_Precision_Recall_Curve, Plot_ROC_Curve,
                           Compare_Multiple_ROC_Curves)

Using TensorFlow backend.


In [0]:
import os
import numpy as np
import pandas as pd
import cv2

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from keras.utils import to_categorical

%matplotlib inline

In [0]:
# fix random seed for reproducibility
seed = 128
np.random.seed(seed)

# Image size for standardisation
image_shape = (150, 150)
input_shape = (-1, 150, 150, 3)

In [0]:
new_test_path = 'images/new_test/'
new_test_list = os.listdir('images/new_test')

nrows = 150
ncolumns = 150
channels = 3
batch_size =64

In [0]:
def Read_and_Process_My_Images(list_of_images, X, y):
    global nrows, ncolumns, new_test_path
    
    for image in list_of_images:
        image=new_test_path+image
            
        #print(image)
        X.append(cv2.resize(cv2.imread(image, cv2.IMREAD_COLOR), \
                            (nrows, ncolumns), interpolation=cv2.INTER_CUBIC))
        y.append(image)
    return X, y

In [0]:
def Read_and_Process_One_Image(image_file):
    global nrows, ncolumns, new_test_path
    
    image=new_test_path+image_file
            
    #print(image)
    X = (cv2.resize(cv2.imread(image, cv2.IMREAD_COLOR), \
                            (nrows, ncolumns), interpolation=cv2.INTER_CUBIC))
    y = image
    return X, y

In [18]:
%cd "{proj_base}"

/gdrive/My Drive/Colab Notebooks/cavy_breed_clf


In [19]:
model, history = Load_NN_Model_Data('inceptionV3_tuned_freeze-230_sgd_v2_epoch-100')
















Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [20]:
%cd "/content"

/content


In [21]:
print()
print(" >> Predicting new test images")
num_images = len(new_test_list)
num_disp_col = 3
num_disp_row = num_images//num_disp_col + (1 if num_images%num_disp_col else 0)
plt.figure(figsize=(20,7*num_disp_row))

for i, image_file in enumerate(new_test_list, start=1):
    X_raw, y = Read_and_Process_One_Image(image_file)
    X = np.array(X_raw)
    y_pred_label, y_val_bool, y_pred = Image_NN_Predict_One(model, X, target_names, verbose=2)
#    print(image_file, y_pred_label)
#    plt.subplot(num_images, 1, i)
    plt.subplot(num_disp_row, num_disp_col, i)
    y_pred_long_desc = "Predicted: {} ({:.0%}), Actual: {}".format(y_pred_label, y_pred[y_val_bool], image_file)
    plt.title(y_pred_long_desc)
    imgplot = plt.imshow(mpimg.imread(new_test_path+image_file))

Output hidden; open in https://colab.research.google.com to view.